# Twin Delayed Deep Deterministic Policy Gradient (TD3)

# Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from tensorboardX import SummaryWriter
from laserhockey.hockey_env import BasicOpponent, HockeyEnv_BasicOpponent
from laserhockey.TrainingHall import TrainingHall
import gym
#import roboschool
import sys
from DDPG.ddpg_agent import DDPGAgent
from DQN.agent import DQNAgent
import matplotlib.pyplot as plt

# Networks

In [2]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

In [3]:
class Actor(nn.Module):
    """Initialize parameters and build model.
        Args:
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            max_action (float): highest action to take
            seed (int): Random seed
            h1_units (int): Number of nodes in first hidden layer
            h2_units (int): Number of nodes in second hidden layer
            
        Return:
            action output of network with tanh activation
    """
    
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, action_dim)

        self.max_action = max_action


    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.max_action * torch.tanh(self.l3(x)) 
        return x



In [4]:
class Critic(nn.Module):
    """Initialize parameters and build model.
        Args:
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            max_action (float): highest action to take
            seed (int): Random seed
            h1_units (int): Number of nodes in first hidden layer
            h2_units (int): Number of nodes in second hidden layer
            
        Return:
            value output of network 
    """
    
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(state_dim + action_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, 1)

        # Q2 architecture
        self.l4 = nn.Linear(state_dim + action_dim, 400)
        self.l5 = nn.Linear(400, 300)
        self.l6 = nn.Linear(300, 1)


    def forward(self, x, u):
        xu = torch.cat([x, u], 1)

        x1 = F.relu(self.l1(xu))
        x1 = F.relu(self.l2(x1))
        x1 = self.l3(x1)

        x2 = F.relu(self.l4(xu))
        x2 = F.relu(self.l5(x2))
        x2 = self.l6(x2)
        return x1, x2


    def Q1(self, x, u):
        xu = torch.cat([x, u], 1)

        x1 = F.relu(self.l1(xu))
        x1 = F.relu(self.l2(x1))
        x1 = self.l3(x1)
        return x1

# Memory

In [5]:
# Code based on: 
# https://github.com/openai/baselines/blob/master/baselines/deepq/replay_buffer.py

# Expects tuples of (state, next_state, action, reward, done)
class ReplayBuffer(object):
    """Buffer to store tuples of experience replay"""
    
    def __init__(self, max_size=150000):
        """
        Args:
            max_size (int): total amount of tuples to store
        """
        
        self.storage = []
        self.max_size = max_size
        self.ptr = 0

    def add(self, data):
        """Add experience tuples to buffer
        
        Args:
            data (tuple): experience replay tuple
        """
        
        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(data)

    def sample(self, batch_size):
        """Samples a random amount of experiences from buffer of batch size
        
        Args:
            batch_size (int): size of sample
        """
        
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        states, actions, next_states, rewards, dones = [], [], [], [], []

        for i in ind: 
            s, a, s_, r, d = self.storage[i]
            states.append(np.array(s, copy=False))
            actions.append(np.array(a, copy=False))
            next_states.append(np.array(s_, copy=False))
            rewards.append(np.array(r, copy=False))
            dones.append(np.array(d, copy=False))

        return np.array(states), np.array(actions), np.array(next_states), np.array(rewards).reshape(-1, 1), np.array(dones).reshape(-1, 1)

# Agent

In [6]:
class TD3(object):
    """Agent class that handles the training of the networks and provides outputs as actions
    
        Args:
            state_dim (int): state size
            action_dim (int): action size
            max_action (float): highest action to take
            device (device): cuda or cpu to process tensors
            env (env): gym environment to use
    
    """
    
    def __init__(self, observation_space, action_space, pretrained=False):
        state_dim  = observation_space.shape[0]
        action_dim = action_space.shape[0]
        max_action = float(action_space.high[0])
        
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=1e-3)

        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=1e-3)

        self.max_action = max_action
        self.action_space = action_space
        
        #self.scaling = torch.tensor([ 5.0,  5.0 , 5.0, 3.0, 3.0, 3.0,  
        #                                2.0,  2.0,  1.0, 1.0, 1.0, 1.0,  
        #                               10.0, 10.0, 10.0, 10.0, 5.0 ,5.0])
        if pretrained:
            self.load(pretrained)


        
    def act(self, state, eps=0.0):
        """Select an appropriate action from the agent policy
        
            Args:
                state (array): current state of environment
                eps (float): how much noise to add to acitons
                
            Returns:
                action (float): action clipped within action range
        
        """
        
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)#*self.scaling
        action = self.actor(state).cpu().data.numpy().flatten()
        if eps != 0: 
            action = (action + np.random.normal(0, eps, size=self.action_space.shape[0]))
            
        return action.clip(self.action_space.low, self.action_space.high)

    
    def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
        """Train and update actor and critic networks
        
            Args:
                replay_buffer (ReplayBuffer): buffer for experience replay
                iterations (int): how many times to run training
                batch_size(int): batch size to sample from replay buffer
                discount (float): discount factor
                tau (float): soft update for main networks to target networks
                
            Return:
                actor_loss (float): loss from actor network
                critic_loss (float): loss from critic network
        
        """
        
        for it in range(iterations):

            # Sample replay buffer 
            x, y, u, r, d = replay_buffer.sample(batch_size)
            state = torch.FloatTensor(x).to(device)#*self.scaling
            action = torch.FloatTensor(u).to(device)
            next_state = torch.FloatTensor(y).to(device)#*self.scaling
            done = torch.FloatTensor(1 - d).to(device)
            reward = torch.FloatTensor(r).to(device)
            
            # #add noise to observation
            #clean_idx = (torch.tensor([ 0,1,2,3,4,5,12,13,14,15, 16, 17]),)
            #noise = torch.FloatTensor(state).data.normal_(0, 1).to(device)
            #noise[clean_idx] = 0
            #state += noise

            # Select action according to policy and add clipped noise 
            noise = torch.FloatTensor(u).data.normal_(0, policy_noise).to(device)
            noise = noise.clamp(-noise_clip, noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)

            # Compute the target Q value
            target_Q1, target_Q2 = self.critic_target(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + (done * discount * target_Q).detach()

            # Get current Q estimates
            current_Q1, current_Q2 = self.critic(state, action)

            # Compute critic loss
            critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q) 

            # Optimize the critic
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

            # Delayed policy updates
            if it % policy_freq == 0:

                # Compute actor loss
                actor_loss = -self.critic.Q1(state, self.actor(state)).mean()

                # Optimize the actor 
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                self.actor_optimizer.step()

                # Update the frozen target models
                for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

                for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)


    def save(self, filename, directory):
        torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
        torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))


    def load(self, filename="best_avg", directory="TD3/saves"):
        self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
        self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

# Runner

In [7]:
class Runner():
    """Carries out the environment steps and adds experiences to memory"""
    
    def __init__(self, env, agent, replay_buffer):
        
        self.env = env
        self.agent = agent
        self.replay_buffer = replay_buffer
        self.obs = env.reset()
        self.done = False
        
    def next_step(self, episode_timesteps, noise=0.1):
        
        action = self.agent.act(np.array(self.obs), eps=0.1)

        # Perform action
        new_obs, reward, done, info = self.env.step(action) 
        done_bool = 0 if episode_timesteps + 1 == 255 else float(done)
        
        #add proxy
        reward += sum(list(info.values()))
        
        # Store data in replay buffer
        replay_buffer.add((self.obs, new_obs, action, reward, done_bool))
        
        self.obs = new_obs
        
        if done:
            self.obs = self.env.reset()
            done = False
            
            return reward, True
        
        return reward, done

# Evaluate

In [8]:
def evaluate_policy(policy, env, eval_episodes=100,render=False):
    """run several episodes using the best agent policy
        
        Args:
            policy (agent): agent to evaluate
            env (env): gym environment
            eval_episodes (int): how many test episodes to run
            render (bool): show training
        
        Returns:
            avg_reward (float): average reward over the number of evaluations
    
    """
    
    avg_reward = 0.
    for i in range(eval_episodes):
        obs = env.reset()
        done = False
        while not done:
            if render:
                env.render()
            action = policy.act(np.array(obs), eps=0)
            obs, reward, done, _ = env.step(action)
            avg_reward += reward

    avg_reward /= eval_episodes

    print("\n---------------------------------------")
    print("Evaluation over {:d} episodes: {:f}" .format(eval_episodes, avg_reward))
    print("---------------------------------------")
    return avg_reward

# Observation

In [9]:
def observe(env,replay_buffer, observation_steps):
    """run episodes while taking random actions and filling replay_buffer
    
        Args:
            env (env): gym environment
            replay_buffer(ReplayBuffer): buffer to store experience replay
            observation_steps (int): how many steps to observe for
    
    """
    
    time_steps = 0
    obs = env.reset()
    done = False

    while time_steps < observation_steps:
        action = env.action_space.sample()
        new_obs, reward, done, _ = env.step(action)

        replay_buffer.add((obs, new_obs, action, reward, done))

        obs = new_obs
        time_steps += 1

        if done:
            obs = env.reset()
            done = False

        print("\rPopulating Buffer {}/{}.".format(time_steps, observation_steps), end="")
        sys.stdout.flush()

# Train

In [10]:
def train(agent, env):
    """Train the agent for exploration steps
    
        Args:
            agent (Agent): agent to use
            env (environment): gym environment
            writer (SummaryWriter): tensorboard writer
            exploration (int): how many training steps to run
    
    """

    total_timesteps = 0
    timesteps_since_eval = 0
    episode_num = 0
    episode_reward = 0
    episode_timesteps = 0
    done = False 
    evaluations = []
    rewards = []
    best_avg = -5
    
    writer = SummaryWriter(comment="-last")
    
    while total_timesteps < EXPLORATION:
    
        if done and total_timesteps != 0:

            rewards.append(episode_reward)
            avg_reward = np.mean(rewards[-100:])

            writer.add_scalar("avg_reward", avg_reward, total_timesteps)
            writer.add_scalar("reward_step", reward, total_timesteps)
            writer.add_scalar("episode_reward", episode_reward, total_timesteps)

            if best_avg < avg_reward:
                best_avg = avg_reward
                print("saving best model....\n")
                agent.save("lasttry","TD3/saves")

            print("\rTotal T: {:d} Episode Num: {:d} Reward: {:f} Avg Reward: {:f}".format(
                total_timesteps, episode_num, episode_reward, avg_reward), end="")
            sys.stdout.flush()


            if avg_reward >= REWARD_THRESH:
                break

            agent.train(replay_buffer, 
                        episode_timesteps, 
                        BATCH_SIZE, GAMMA, TAU, NOISE, NOISE_CLIP, POLICY_FREQUENCY)

            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1 

        reward, done = runner.next_step(episode_timesteps)
        episode_reward += reward

        episode_timesteps += 1
        total_timesteps += 1
        timesteps_since_eval += 1

# Config

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEED = 5
OBSERVATION = 10000
EXPLORATION = 1000000
BATCH_SIZE = 100
GAMMA = 0.99
TAU = 0.005
NOISE = 0.2
NOISE_CLIP = 0.5
EXPLORE_NOISE = 0.2
POLICY_FREQUENCY = 2
EVAL_FREQUENCY = 5000
REWARD_THRESH = 100

In [12]:
env = HockeyEnv_BasicOpponent(weak_opponent=False)

basic  = BasicOpponent(weak=False)
td3   = TD3(env.observation_space, env.action_space, pretrained='stronger')
loser = TD3(env.observation_space, env.action_space, pretrained='smallhall')
q_agent = DQNAgent(env.observation_space, 
                         env.discrete_action_space,
                        convert_func =  env.discrete_to_continous_action,
                        pretrained   = 'DQN/weights/training_hall_1')

#env.register_opponents([basic])#,ddpg,q_agent])

# Main

In [13]:
# Set seeds
env.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

policy = TD3(env.observation_space, env.action_space, pretrained=False)

replay_buffer = ReplayBuffer(max_size=300000)

runner = Runner(env, policy, replay_buffer)

total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True

In [14]:
# Populate replay buffer
observe(env, replay_buffer, OBSERVATION)

Populating Buffer 10000/10000.

In [ ]:
# Train agent
train(policy, env)

Total T: 23429 Episode Num: 139 Reward: -9.761105 Avg Reward: -5.23951951saving best model....

Total T: 23482 Episode Num: 140 Reward: 25.098248 Avg Reward: -4.961365saving best model....

Total T: 27543 Episode Num: 164 Reward: -0.361061 Avg Reward: -4.8943554saving best model....

Total T: 27781 Episode Num: 166 Reward: -13.512818 Avg Reward: -4.666807saving best model....

Total T: 28283 Episode Num: 168 Reward: -8.939579 Avg Reward: -4.5676238saving best model....

Total T: 28534 Episode Num: 169 Reward: -1.852731 Avg Reward: -4.353128saving best model....

Total T: 28584 Episode Num: 170 Reward: 24.126631 Avg Reward: -3.972783saving best model....

Total T: 28618 Episode Num: 171 Reward: 24.881277 Avg Reward: -3.496415saving best model....

Total T: 29198 Episode Num: 178 Reward: -13.449070 Avg Reward: -3.771616saving best model....

Total T: 29455 Episode Num: 180 Reward: -13.023944 Avg Reward: -3.348651saving best model....

Total T: 29539 Episode Num: 181 Reward: -9.749664 Avg

Total T: 168663 Episode Num: 1242 Reward: 20.195531 Avg Reward: 10.139022saving best model....

Total T: 168694 Episode Num: 1243 Reward: 24.764423 Avg Reward: 10.446942saving best model....

Total T: 168786 Episode Num: 1244 Reward: 23.787068 Avg Reward: 10.811367saving best model....

Total T: 168853 Episode Num: 1245 Reward: 27.498815 Avg Reward: 10.825773saving best model....

Total T: 168911 Episode Num: 1246 Reward: 24.524173 Avg Reward: 10.922100saving best model....

Total T: 168941 Episode Num: 1247 Reward: 25.614533 Avg Reward: 11.163769saving best model....

Total T: 168986 Episode Num: 1248 Reward: 24.465771 Avg Reward: 11.540386saving best model....

Total T: 169395 Episode Num: 1250 Reward: -3.781459 Avg Reward: 11.536050saving best model....

Total T: 169449 Episode Num: 1251 Reward: 29.121929 Avg Reward: 11.794942saving best model....

Total T: 169591 Episode Num: 1252 Reward: 26.373674 Avg Reward: 11.832285saving best model....

Total T: 169901 Episode Num: 1254 Reward

Total T: 403502 Episode Num: 3395 Reward: 25.637483 Avg Reward: 22.914925saving best model....

Total T: 403549 Episode Num: 3396 Reward: 24.523169 Avg Reward: 23.149955saving best model....

Total T: 460423 Episode Num: 4183 Reward: 25.926721 Avg Reward: 23.0259954saving best model....

Total T: 460466 Episode Num: 4184 Reward: 24.627376 Avg Reward: 23.237855saving best model....

Total T: 460502 Episode Num: 4185 Reward: 26.297546 Avg Reward: 23.244775saving best model....

Total T: 468793 Episode Num: 4312 Reward: 27.464144 Avg Reward: 23.6493561saving best model....

Total T: 469518 Episode Num: 4325 Reward: 25.969783 Avg Reward: 23.5447709saving best model....

Total T: 469777 Episode Num: 4331 Reward: 26.414117 Avg Reward: 23.854595saving best model....

Total T: 473909 Episode Num: 4396 Reward: 24.435295 Avg Reward: 21.6095877

In [ ]:
policy.save(filename="last", directory="TD3/saves")

evaluate_policy(policy, env, eval_episodes=100, render=False)

In [ ]:
env.update_weights(add=True)
stats = np.array(env.weights).T
x,y = stats.shape
plt.figure(figsize=(16,5))
for i,agent,name in zip(stats,env.agents,['weak','strong']):
    plt.plot(np.arange(y-1)*50,i[1:],label=name)
plt.legend()
plt.xlabel('Episodes')
plt.ylabel('Winscore')
plt.savefig('TD3/Plots/last.svg')
plt.show()

In [ ]:
env.close()